In [0]:
import pyspark.sql.functions as sf
from time import time
spark.conf.set("spark.sql.session.timeZone", "America/New_York")

In [0]:
# Read data from 'runescape.02_silver.item_mapping'
df_map = spark.read.table("runescape.02_silver.item_mapping")

In [0]:
%skip
df_map.display()

In [0]:
# Read data from 'runescape.02_silver.latest_prices_cleansed'
# and filter to the last 24 hours
unix_time_24_hours_ago = time() - 86400
df = spark.read.table("runescape.02_silver.latest_prices_enriched").filter(f"time >= {unix_time_24_hours_ago}")
df = df.withColumn("NYCTime", sf.from_unixtime('time'))
df.count()

In [0]:
# Read data from 'runescape.01_bronze.1h_prices' to get Volumes
# TODO replace this read with the new latest_1_prices table that will have the latest 1 hour price for each item
df_avg = spark.read.table("runescape.01_bronze.1h_prices")

In [0]:
# Get average of 1 hour price data over the last 24 hours
# TODO replace this with actual 24 hour calculation based on underlying raw data
df_24_hours_avg = df_avg.\
    filter(f"time >= {unix_time_24_hours_ago}").\
    groupBy("id").\
    agg(sf.round(sf.avg("avg1HourHigh"), 2).alias("avg_24hr_High")
    , sf.round(sf.avg("avg1HourHighVolume"), 2).alias("avg_24hr_High_Volume")
    , sf.round(sf.avg("avg1HourLow"), 2).alias("avg_24hr_Low")
    , sf.round(sf.avg("avg1HourLowVolume"), 2).alias("avg_24hr_Low_Volume"))




In [0]:
%skip

# Filter to only use the most recent Volume data
max_time = df_avg.select(sf.max(df_avg.time))

#df_avg = df_avg.filter("time == max_time.first('time'")
df_avg = df_avg.join(max_time, df_avg.time == sf.col("max(time)"))
df_avg = df_avg.drop("max(time)").drop("time")
#df_avg.display()




In [0]:
# filter out low limit items
df2 = df.filter(sf.col("limit") >= 70)
# filter to only sell data
df2 = df2.filter(sf.col("highorlow") == "high")

In [0]:

df3 = df2.withColumn("high_Alch_Margin", sf.round((sf.col("highalch") - sf.col("price")) - 100,1))
# Join with 24 hour data to get volume
df3 = df3.join(df_24_hours_avg, "id")
df3 = df3.filter("avg_24hr_High_Volume >= 100")
df3 = df3.filter("high_Alch_Margin >= 400")
df3 = df3.sort("high_Alch_Margin", ascending=False)
df3.display()

In [0]:
time_window = time() - 7200 # 2hours
df4 = df3.filter(f"time >= {time_window}")
df4 = df4.sort("high_Alch_Margin", ascending=False)
df4.display()

In [0]:
# make an aggregate for high aclh margins

In [0]:
%skip
df10 = df9.groupBy("id","highorlow").agg(sf.round(sf.avg("price"), 2).alias("avg_price"), sf.round(sf.std("price"), 2).alias("std_price"))
#join with volume data
df10 = df10.join(df_avg, "id").withColumn("std/avg", sf.round(sf.col("std_price")/sf.col("avg_price"), 3))
df10 = df10.join(df_map,"id")
df10 = df10.withColumn("high_Alch_Margin", sf.round((sf.col("highalch") - sf.col("avg_price")) - 100,1))
df10 = df10.withColumn("price-1hour_high_price", sf.round(sf.col("avg_price") - sf.col("avg1HourHigh") , 2))
df10 = df10.withColumn("limit_X_std_price", sf.col("std_price")*sf.col("limit"))



df10 = df10.filter("highorlow = 'low'")
df10 = df10.filter("std_price >= 2")
df10 = df10.filter("avg1HourHighVolume > 1000")
df10 = df10.sort("std/avg", ascending=False)
df10.display()

In [0]:
%skip
df8.display()